# Q-Synth API Tutorial

This tutorial demonstrates the main features and usage of the Q-Synth API for optimal quantum circuit synthesis. 

We assume you have already installed the Q-Synth package. If not, Q-Synth can be installed using the following pip-command. You probably want to do that in your own virtual environment (with venv):

```
pip install Q-Synth
```
---

## Basic Input, Output, Arguments

### Quantum Circuit
Loading a quantum circuit from a file in OPENQASM 2.0 format:

In [ ]:
from qiskit import QuantumCircuit
ecai24 = QuantumCircuit.from_qasm_file('ecai24.qasm')
print(ecai24)

Creating a quantum circuit using Qiskit:

In [ ]:
from qiskit import QuantumCircuit
def qc_example():
    qc = QuantumCircuit(3)
    qc.cx(0, 1)
    qc.s(0)
    qc.cx(0, 2)
    qc.cx(1, 2)
    return qc
print(qc_example())

### Specifying a Quantum Platform

For layout mapping and layout-aware synthesis, the coupling graph of the platform needs to be specified. It will be bi-directional by default. There are three methods:

1. Specifying the coupling graph explicitly:

In [ ]:
from qsynth import get_coupling_graph
# Custom coupling graph
coupling_graph = get_coupling_graph(coupling_graph=[[0,1],[1,2]], bidirectional=1)
print(coupling_graph)

2. Specifying a predefined platforms (e.g., 'tenerife', 'melbourne', 'sycamore', 'rigetti-80', 'eagle')

In [ ]:
# Predefined platform example (e.g., 'tenerife')
tenerife = get_coupling_graph(platform='tenerife')
print(tenerife)

3. Generating an instance of a scalable platform (e.g. line-N, cycle-N, grid-N)

In [ ]:
# Scalable platform example (e.g., 'line-4' for nearest neighbor line of 4 qubits)
line4 = get_coupling_graph(platform='line-4')
print(line4)

### Understanding Q-Synth output: MappedResult
The output of Q-Synth synthesis functions is a `MappedResult` object containing:
- `circuit`: the mapped/optimized quantum circuit
- `initial_mapping`: the initial mapping of qubits
- `final_mapping`: the final mapping of qubits after synthesis

In [ ]:
from qsynth import layout_synthesis
qc = qc_example()
result = layout_synthesis(circuit=qc, coupling_graph=line4, verbose=-1)
print('Mapped circuit:')
print(result.circuit)
print('Initial mapping:', result.initial_mapping)
print('Final mapping:', result.final_mapping)

### Optimization metrics
Q-Synth supports several optimization metrics:
- CNOT count (`cx-count`) : Computes optimal Number of CNOT gates
- CNOT depth (`cx-depth`) : Computes optimal CNOT depth
- CNOT count + CNOT depth (`cx-count_cx-depth`) : First computes optimal CNOT count and then optimizes CNOT depth without increasing CNOT count.
- CNOT depth + CNOT count (`cx-depth_cx-count`) : First computes optimal CNOT depth and then optimizes CNOT count without increasing CNOT depth.

## Layout Synthesis

### Minimal example:

In [ ]:
# Minimal layout synthesis example
from qsynth import layout_synthesis, get_coupling_graph
from qiskit import QuantumCircuit
qc = qc_example()
coupling_graph = get_coupling_graph(coupling_graph=[[0,1],[1,2]], bidirectional=1)
result = layout_synthesis(circuit=qc, coupling_graph=coupling_graph, metric='cx-count', verbose=-1)
print('Mapped circuit:')
print(result.circuit)
print('Initial mapping:', result.initial_mapping)
print('Final mapping:', result.final_mapping)

### Using subarchitectures for large platforms:

When mapping to large platforms, we can use subarchitectures.  
By default, Q-Synth maps to all maximal subarchitectures with 0 ancillas.  
Note that we write the resulting circuit to a file.


In [ ]:
from qsynth import layout_synthesis, get_coupling_graph
from qiskit import QuantumCircuit, qasm2

barenco = QuantumCircuit.from_qasm_file('barenco_tof_3.qasm')
sycamore = get_coupling_graph(platform='sycamore')
result = layout_synthesis(circuit=barenco, coupling_graph=sycamore, metric='cx-count', subarchitecture=True, verbose=0)
qasm2.dump(result.circuit, "mapped_barenco_tof_3.qasm")

We can also specify the number of ancillas to use with subarchitectures:

In [ ]:
result = layout_synthesis(circuit=barenco, coupling_graph=sycamore, metric='cx-count', subarchitecture=True, num_ancillary_qubits=1)

### Optimal layout synthesis for a given initial mapping

You can provide Q-Synth with an initial mapping, for instance one computed using Qiskit's SABRE heuristics. 
Q-Synth will then compute the minimally mapped layout for that initial mapping.

In [ ]:
# Example: Using Qiskit for initial mapping, then Q-Synth for optimal synthesis from that mapping
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import SabreLayout
vqe = QuantumCircuit.from_qasm_file('vqe_8_3_5_100.qasm')
sycamore = get_coupling_graph(platform='sycamore')
layout_pass = SabreLayout(CouplingMap(sycamore), seed=1)
mapped_vqe = PassManager(layout_pass).run(vqe)
print("Qiskit SabreLayout mapped circuit counts:", mapped_vqe.count_ops())
initial_layout = dict(enumerate(mapped_vqe.layout.initial_index_layout(filter_ancillas=True)))
print('Initial layout from Qiskit SabreLayout:', initial_layout)
result = layout_synthesis(circuit=vqe, coupling_graph=sycamore, metric='cx-count', initial_mapping=initial_layout, verbose=0)
print("Q-Synth mapped circuit counts:", result.circuit.count_ops())
print('Initial Mapping:', result.initial_mapping)
print('Final mapping:', result.final_mapping)


Using Q-Synth for the initial mapping computed by SabreLayout gives reduces the number of swaps from 9 to 8 (optimal for this initial mapping)

## CNOT Synthesis

CNOT synthesis applies to pure CNOT circuits, but can be applied to CNOT slices as well (see Peephole synthesis below).
Here we optimize for `cx-count`, but other metrics are supported as well.

We now explain the four basic variants: S, W, S+R, W+R. Note that
We will illustrate them on this example from the ECAI 2024-paper:

In [ ]:
from qiskit import QuantumCircuit
qc = QuantumCircuit.from_qasm_file('ecai24.qasm')
print('Original circuit:')
print(qc)

### Pure CNOT synthesis

1. Minimal example (S): strict equality, no layout restrictions

In [ ]:
# Minimal CNOT synthesis example (S)
from qsynth import cnot_synthesis
result = cnot_synthesis(circuit=qc, metric='cx-count', verbose=-1)
print('Optimized circuit:')
print(result.circuit)

2. Allowing output qubit permutation (W): weak equality, no layout restrictions

    The result can be further reduced by allowing a permutation on the output qubits:

In [ ]:
# CNOT synthesis with output qubit permutation (W)
from qsynth import cnot_synthesis
result = cnot_synthesis(circuit=qc, metric='cx-count', output_qubit_permute=True, verbose=-1)
print('Optimized circuit:')
print(result.circuit)
print('Final mapping:', result.final_mapping)

### Layout-aware CNOT synthesis

3. Layout aware synthesis (S+R): strict equality, layout restrictions

    Layout restrictions can be specified by just providing a `coupling_graph`.
    Here we define the nearest neighbour platform on 4 qubits as `line-4`.

In [ ]:
# Layout aware CNOT synthesis (S+R)
from qsynth import cnot_synthesis, get_coupling_graph
coupling_graph = get_coupling_graph(platform="line-4", bidirectional=1)
print("Coupling graph: ", coupling_graph)
result = cnot_synthesis(circuit=qc, metric='cx-count', coupling_graph=coupling_graph, verbose=-1)
print('Optimized circuit:')
print(result.circuit)

4. Layout aware with output qubit permutation (W+R): weak equivalence, with layout restrictions

    Again, the result can be reduced by allowing a permutation on the output qubits.

In [ ]:
# Layout aware CNOT synthesis with output qubit permutation (W+R)
from qsynth import cnot_synthesis, get_coupling_graph

result = cnot_synthesis(circuit=qc, metric='cx-count', coupling_graph=coupling_graph, output_qubit_permute=True, verbose=-1)
print('Optimized circuit:')
print(result.circuit)
print('Final mapping:', result.final_mapping)

## Clifford Synthesis

Clifford Synthesis can be applied to pure Clifford circuits, recognizing e.g. gates X, Y, Z, H, S, CX, CZ.  
(See also Peephole synthesis for Clifford slices below).

In [ ]:
# Example Clifford Circuit
from qiskit import QuantumCircuit
clifford = QuantumCircuit.from_qasm_file('04q_33936_clifford.qasm')
print('Original circuit:')
print(clifford.draw("text", fold=-1))


Q-Synth can optimize Clifford circuits for `cx_count` and `cx_depth` and other metrics.  
Here we illustrate `cx-depth` and `cx-depth_cx-count` metrics.  
We support three variants: S, W, S+R.

1. Minimal example (S): strict equivalence, no layout restrictions  
    (minimizing for `cx-depth`)

In [ ]:
# Minimal Clifford synthesis example (S)
from qsynth import clifford_synthesis
result = clifford_synthesis(circuit=clifford, metric='cx-depth', verbose=-1)
print('Optimized circuit:')
print(result.circuit.draw("text", fold=-1))

This CNOT-depth optimization has reduced `cx-depth` from 7 to 4 and `cx-count`from 8 to 7.  
To further optimize the `cx-count` without increasing `cx-depth`, we can use the `cx-depth_cx-count` metric:

In [ ]:
# Clifford synthesis example (S) with combined metric
from qsynth import clifford_synthesis
result = clifford_synthesis(circuit=clifford, metric='cx-depth_cx-count', verbose=-1)
print('Optimized circuit:')
print(result.circuit.draw("text", fold=-1))

Note that the `cx-depth` is still 4 but the `cx-count` has been further reduced from 7 to 6.

2. Allowing output qubit permutation (W): weak equivalence

In [ ]:
result = clifford_synthesis(circuit=clifford, metric='cx-depth_cx-count', output_qubit_permute=True, verbose=-1)
print('Optimized circuit:')
print(result.circuit.draw("text", fold=-1))
print('Final mapping:', result.final_mapping)

With output qubit permutations, we can reduce this even further to `cx-depth`=3 and `cx-count`=5.

3. Layout aware synthesis (S+R): strict equivalence, with layout restrictions

In [ ]:
from qsynth import get_coupling_graph
coupling_graph = get_coupling_graph(platform="line-4", bidirectional=1)
result = clifford_synthesis(circuit=clifford, metric='cx-count', coupling_graph=coupling_graph, verbose=0)
print('Optimized circuit:')
print(result.circuit.draw("text", fold=-1))

Now we need 8 CNOTs due to the layout restrictions.

## Peephole Synthesis

Q-Synth can also be used to resynthesize general quantum circuits by slicing them into Clifford or CNOT slices, and resynthesizing each slice optimally.  

In [ ]:
from qsynth import peephole_synthesis
from qiskit import QuantumCircuit
print('Original circuit:')
qc = qc_example()
print(qc)

### CNOT slicing 

Note that the circuit above has 2 CNOT slices, which cannot be optimized:

In [ ]:
result = peephole_synthesis(circuit=qc, slicing='cnot', metric='cx-count', verbose=-1)
print('Optimized circuit:')
print(result.circuit)

### Clifford slicing 

Note, the circuit above has a single Clifford slice, so now there is some room for optimization:

In [ ]:
result = peephole_synthesis(circuit=qc, slicing='clifford', metric='cx-count', verbose=0)
print('Optimized circuit:')
print(result.circuit)

### Resynthesizing general quantum circuits
  
We now illustrate the re-synthesis of a non-Clifford quantum circuit consisting of multiple slices, without layout restrictions:

In [ ]:
barenco = QuantumCircuit.from_qasm_file('barenco_tof_3.qasm')
result = peephole_synthesis(circuit=barenco, slicing='clifford', metric='cx-count', verbose=0)

For layout-aware Peephole Resynthesis, we expect the input circuit to be already layout mapped.
If it is not layout mapped, one could either use Q-Synth or Qiskit to first perform layout mapping and then do peephole re-synthesis.

Here is an example of layout-aware peephole synthesis:

In [ ]:
melbourne = get_coupling_graph(platform="melbourne")
mapped_result = layout_synthesis(circuit=barenco, coupling_graph=melbourne, metric='cx-count', verbose=0)
resynthesized_result = peephole_synthesis(circuit=mapped_result.circuit, slicing='clifford', metric='cx-count', coupling_graph=melbourne, verbose=0)

Optimal layout synthesis adds 7 swaps, resulting in total 45 CNOTs.
Applying layout-aware peephole Resynthesis reduces the cx-count 45 -> 42 and cx-depth 43 -> 39.